In [57]:
import codecs,re,pickle
import pandas as pd
import itertools as it

In [58]:
trainingDataFile="/Users/gilles/Github/sigmorphon2016/data/german-task2-train"
cat="V"
nomLexique="/Users/gilles/Box Sync/2015-Data/German-%s.pkl"%cat

In [59]:
with codecs.open(trainingDataFile,"r",encoding="utf8") as input:
    trainingData=input.readlines()

In [60]:
inflectedPairs=[]
for line in trainingData:
    (attributes1,form1,attributes2,form2)=line.strip().split("\t")
    (pos1,atts1)=attributes1.split(",",1)
    (pos2,atts2)=attributes2.split(",",1)
    inflectedPairs.append([(pos1,atts1,form1),(pos2,atts2,form2)])

In [61]:
inflectedPairs

[[(u'pos=ADJ', u'case=ACC,comp=CMPR,gen=FEM,num=SG', u'aerodynamischere'),
  (u'pos=ADJ', u'case=ACC,comp=SPRL,gen=NEUT,num=SG', u'aerodynamischstes')],
 [(u'pos=ADJ', u'case=ACC,comp=CMPR,gen=FEM,num=SG', u'aktivere'),
  (u'pos=ADJ', u'case=NOM,comp=SPRL,gen=NEUT,num=SG', u'aktivstes')],
 [(u'pos=ADJ', u'case=ACC,comp=CMPR,gen=FEM,num=SG', u'ambitioniertere'),
  (u'pos=ADJ', u'case=GEN,gen=FEM,num=SG', u'ambitionierter')],
 [(u'pos=ADJ', u'case=ACC,comp=CMPR,gen=FEM,num=SG', u'aufnahmef\xe4higere'),
  (u'pos=ADJ', u'case=DAT,comp=CMPR,gen=FEM,num=SG', u'aufnahmef\xe4higerer')],
 [(u'pos=ADJ', u'case=ACC,comp=CMPR,gen=FEM,num=SG', u'aufrechtere'),
  (u'pos=ADJ', u'case=ACC,comp=SPRL,gen=FEM,num=SG', u'aufrechtste')],
 [(u'pos=ADJ', u'case=ACC,comp=CMPR,gen=FEM,num=SG', u'ausdruckslosere'),
  (u'pos=ADJ', u'case=GEN,comp=CMPR,gen=FEM,num=SG', u'ausdrucksloserer')],
 [(u'pos=ADJ', u'case=ACC,comp=CMPR,gen=FEM,num=SG', u'ausgegrautere'),
  (u'pos=ADJ', u'case=ACC,comp=SPRL,num=PL', u'ausg

In [62]:
def listAVs(chaine):
    dictAVs={}
    avs=chaine.split(",")
    for av in avs:
        attribute,value=av.split("=")
        m=re.match(ur"{(.+)\/(.+)}",value)
        if m:
            dictAVs[attribute]=set([m.group(1),m.group(2)])
        else:
            dictAVs[attribute]=set([value])
    return dictAVs

In [63]:
attributes={}
dimensions=set()
inflectedVerbs=[]
for c1,c2 in inflectedPairs:
    if cat in c1[0] and cat in c2[0]:
        inflectedVerbs.append((c1,c2))
        avs=listAVs(c1[1])
        for av in avs:
#            if "gen" in av: print c1, c2
            dimensions.add(av)
            if not av in attributes:
                attributes[av]=set()
            attributes[av]|=avs[av]
        avs=listAVs(c2[1])
        for av in avs:
#            if "gen" in av: print c2, c1
            dimensions.add(av)
            if not av in attributes:
                attributes[av]=set()
            attributes[av]|=avs[av]
repForme=[u"form"]+list(dimensions)

In [64]:
lstColumns=[a+"1" for a in repForme]+[a+"2" for a in repForme]
pdPaires=pd.DataFrame(columns=lstColumns)
pdFormes=pd.DataFrame(columns=repForme)

In [ ]:
num=0
for c1,c2 in inflectedVerbs:
    avs=listAVs(c1[1])
    pdAVs={av:list(avs[av]) for av in avs}
    pdAVs[u"form"]=[c1[2]]
    produitAVs=list(it.product(*pdAVs.values()))
    for produitAV in produitAVs:
        dictLigne={a:[b] for (a,b) in zip(pdAVs.keys(),produitAV)}
        pdLigne=pd.DataFrame.from_dict(dictLigne)
        pdFormes=pd.concat([pdFormes,pdLigne],ignore_index=True)
    avs=listAVs(c2[1])
    pdAVs={av:list(avs[av]) for av in avs}
    pdAVs[u"form"]=[c2[2]]
    produitAVs=list(it.product(*pdAVs.values()))
    for produitAV in produitAVs:
#        print produitAV
        dictLigne={a:[b] for (a,b) in zip(pdAVs.keys(),produitAV)}
        pdLigne=pd.DataFrame.from_dict(dictLigne)
        pdFormes=pd.concat([pdFormes,pdLigne],ignore_index=True)

In [82]:
pdFormes

,aspect,finite,form,mood,num,per,tense
0,NaN,NFIN,aktualisieren,NaN,NaN,NaN,NaN
1,PFV,NaN,aktualisierten,SBJV,PL,3,PST
2,PFV,NaN,aktualisierten,COND,PL,3,PST
3,NaN,NFIN,bedanken,NaN,NaN,NaN,NaN
4,PFV,NaN,bedankten,SBJV,PL,3,PST
5,PFV,NaN,bedankten,COND,PL,3,PST
6,NaN,NFIN,beerdigen,NaN,NaN,NaN,NaN
7,PFV,NaN,beerdigte,SBJV,SG,3,PST
8,PFV,NaN,beerdigte,COND,SG,3,PST
9,NaN,NFIN,belohnen,NaN,NaN,NaN,NaN


In [66]:
listFormes=[]
for c1,c2 in inflectedVerbs:
    avs=listAVs(c1[1])
    pdAVs={av:avs[av] for av in avs}
    produitAVs=list(it.product(*pdAVs.values()))
    for produitAV in produitAVs:
        listFormes.append([c1[2],c1[2],"x".join(produitAV)])
    avs=listAVs(c2[1])
    pdAVs={av:avs[av] for av in avs}
    produitAVs=list(it.product(*pdAVs.values()))
    for produitAV in produitAVs:
        listFormes.append([c1[2],c2[2],"x".join(produitAV)])

In [67]:
pdLexique=pd.DataFrame(listFormes,columns=[u"clé","forme","case"])
pdLexique.columns=["lexeme","phono","case"]
pdLexique=pdLexique.drop_duplicates()

In [68]:
listLexeme=pdLexique["lexeme"].unique().tolist()
listPhono=pdLexique["phono"].unique().tolist()
pdChange=pdLexique[(pdLexique["phono"].isin(listLexeme)) & (pdLexique["phono"]!=pdLexique["lexeme"])][["lexeme","phono"]]

In [69]:
def findChange(lexeme,vus=[]):
    print lexeme
    if lexeme in dictChange and not lexeme in vus:
        if dictChange[lexeme] in dictChange:
            vus.append(lexeme)
            return findChange(dictChange[lexeme],vus)
        else:
            return dictChange[lexeme]
    else:
        return lexeme
    
dictChange=dict(zip(pdChange.phono,pdChange.lexeme))
for change in dictChange:
    dictChange[change]=findChange(change)
dictChange

karikiere
foultest
kumulierte
schweige
komponieren
lügen
begrüsse
rietet
riete
liehen
friert
listete
niete
rechne
erwarbst
geprahlt
könnten
stockten
gekennzeichnet
tropfte
transferiertest
verbautet
dokumentiertet
spross
verantworteten
drücktet
parkettiertest
gelieren
erlabest
währest
variieren
schmiertest
versuchen
fragmentiert
vermintest
kicken
kicktet
röchest
erreicht
kriminalisierest
bestritt
widerlegst
dressierten
dressiere
eiltet
konfiszierst
fieptet
schweigest
blanchierst
goss
gießen
goss
spülten
kasernierten
reifizierest
reifizieren
reifizierest
schmeckten
benützt
verurteilt
scannst
enthielten
härtetest
vermögen
vermochte
spucke
trenne
annullieren
spaziert
spazieren
spaziert
sorgten
entfolgen
fuhren
ziehen
reformieren
obfuskiere
versandtest
balkanisierte
bersten
priorisierte
zeugen
balle
repräsentieren
kopulieren
wappne
zielen
zielt
balltest
balle
verewigtest
drohen
drohten
drohen
gesprossen
spross
kommuniziert
machten
machend
machten
hassen
überdenken
überdenkt
reizte
reizt
rei

{u'aaltest': u'aalten',
 u'absolvierte': u'absolvierten',
 u'alarmiere': u'alarmierte',
 u'aluminierten': u'aluminierest',
 u'annektiere': u'annektiertet',
 u'annektierten': u'annektiertet',
 u'annullieren': u'annullierest',
 u'antichambrieret': u'antichambrierest',
 u'apportiert': u'apportiertet',
 u'arbeitete': u'arbeite',
 u'argumentieren': u'argumentiertet',
 u'arrangieren': u'arrangierest',
 u'auserkoren': u'auserkorst',
 u'balkanisierte': u'balkanisierst',
 u'balle': u'ballten',
 u'balltest': u'ballten',
 u'bandest': u'binde',
 u'barg': u'bargst',
 u'beachte': u'beachtetet',
 u'beachtetest': u'beachtetet',
 u'bearbeitete': u'bearbeitend',
 u'bearbeitetet': u'bearbeite',
 u'beaufsichtigt': u'beaufsichtigten',
 u'beeintr\xe4chtigtet': u'beeintr\xe4chtigt',
 u'beerdigen': u'beerdigten',
 u'befielen': u'befielet',
 u'begr\xfcsse': u'begr\xfcssest',
 u'beg\xfcnstigte': u'beg\xfcnstigest',
 u'bekr\xe4nzen': u'bekr\xe4nzet',
 u'bekundet': u'bekundete',
 u'benenntet': u'benannten',
 u'be

In [79]:
pdLexique.groupby(["phono","case"]).agg(",".join)

lexeme
phono          case                                         
aalen          PLxPRSxINDx1                          aaltest
               PLxPRSxINDx3                           aalten
aalend         PRS                                      aalt
aalt           PLxPRSxINDx2                             aalt
aalte          SGxPSTxINDxPFVx3                       aaltet
aalten         PLxPSTxCONDxPFVx1                      aalten
               PLxPSTxINDxPFVx3                       aalten
               PLxPSTxSBJVxPFVx1                      aalten
aaltest        SGxPSTxINDxPFVx2               aaltest,aalten
aaltet         PLxPSTxCONDxPFVx2                      aaltet
               PLxPSTxSBJVxPFVx2                      aaltet
aasen          PLxPRSxINDx1                            aasen
aastet         PLxPSTxINDxPFVx2                        aasen
abalienieren   PLxPRSxINDx3                     abalienieret
abalienieret   PLxPRSxOPTx2                     abalienieret
               PLxPRSxSBJVx2                    abalienieret
abandonniere   SGxPRSxOPTx3                    abandonnieren
               SGxPRSxSBJVx3                   abandonnieren
abandonnieren  PLxPRSxOPTx3                    abandonnieren
               PLxPRSxSBJVx3                   abandonnieren
abandonnierend PRS                            abandonnierend
abandonnierten PLxPSTxCONDxPFVx1              abandonnierend
               PLxPSTxSBJVxPFVx1              abandonnierend
abonniere      SGxPRSxOPTx1                      abonnierend
               SGxPRSxSBJVx1                     abonnierend
abonnierend    PRS                               abonnierend
abonnierte     SGxPSTxINDxPFVx1                  abonniertet
abonniertet    PLxPSTxINDxPFVx2                  abonniertet
absolvieren    NFIN                              absolvierst
               PLxPRSxOPTx1                      absolvierte
...                                                      ...
überwachtest   SGxPSTxINDxPFVx2                  überwachend
überwand       SGxPSTxINDxPFVx3                   überwandet
überwanden     PLxPSTxINDxPFVx3                   überwändet
überwandet     PLxPSTxINDxPFVx2                   überwandet
überweisen     PLxPRSxOPTx3                         überwies
               PLxPRSxSBJVx3                        überwies
überwies       SGxPSTxINDxPFVx1                     überwies
überwinden     PLxPRSxOPTx1                       überwunden
               PLxPRSxSBJVx1                      überwunden
überwindest    SGxPRSxOPTx2                        überwände
               SGxPRSxSBJVx2                       überwände
überwunden     PST                                überwunden
überwältigen   PLxPRSxOPTx3       überwältigest,überwältigen
               PLxPRSxSBJVx3      überwältigest,überwältigen
überwältigest  SGxPRSxOPTx2                    überwältigest
               SGxPRSxSBJVx2                   überwältigest
überwältigte   SGxPSTxCONDxPFVx1                überwältigen
               SGxPSTxSBJVxPFVx1                überwältigen
überwände      SGxPSTxCONDxPFVx1                   überwände
               SGxPSTxSBJVxPFVx1                   überwände
überwändet     PLxPSTxCONDxPFVx2                  überwändet
               PLxPSTxSBJVxPFVx2                  überwändet
überzeugen     PLxPRSxINDx1                     überzeugtest
überzeugest    SGxPRSxOPTx2                      überzeugest
               SGxPRSxSBJVx2                     überzeugest
überzeugst     SGxPRSxINDx2                       überzeugst
überzeugten    PLxPSTxINDxPFVx3                   überzeugst
überzeugtest   SGxPSTxCONDxPFVx2                überzeugtest
               SGxPSTxSBJVxPFVx2                überzeugtest
überzeugtet    PLxPSTxINDxPFVx2                  überzeugest

[10025 rows x 1 columns]

In [50]:
listChange=pdLexique[(pdLexique["phono"].isin(listLexeme)) & (pdLexique["phono"]!=pdLexique["lexeme"])]["phono"].unique().tolist()
pdLexique.loc[(pdLexique["lexeme"].isin(dictChange)),"lexeme"]

6            beerdigen
7            beerdigen
8            beerdigen
19          brillieren
20          brillieren
39              ernten
40              ernten
41              ernten
54              helfen
55              helfen
59       improvisieren
60       improvisieren
70         komponieren
71         komponieren
72         komponieren
84              nennen
85              nennen
88          plissieren
89          plissieren
122             treten
123             treten
124             treten
149        annullieren
150        annullieren
162           bepissen
163           bepissen
164           bepissen
167            berufen
168            berufen
169            berufen
             ...      
9766     kompostierend
9767     kompostierend
9772          kreisend
9773          kreisend
9774          kreisend
9861       plissierend
9862       plissierend
9863       plissierend
10230       enthauptet
10231       enthauptet
10244          erteilt
10245          erteilt
10256      

In [11]:
german1graphe=(u"üöäwc",u"y2Evk")
german2=[u"sch",u"ch",u"sc",u"x",u"ß",u"ck",u"#v",u"#s",u"uç",u"oç",u"aç",u"e#",u"qu",u"ci",u"ce"]
bdlexique2=[u"S",u"ç",u"sk",u"ks",u"ss",u"k",u"#f",u"#z",u"ux",u"ox",u"ax",u"6#",u"ku",u"si",u"se"]
germanIn = unicode(german1graphe[0])
germanNum = [ord(char) for char in germanIn]
bdlexiqueOut = unicode(german1graphe[1])
neutralise = dict(zip(germanNum, bdlexiqueOut))

In [12]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp="#"+unicode(chaine.decode('utf8'))+"#"
        for numBigramme,bigramme in enumerate(german2):
            temp=temp.replace(bigramme,bdlexique2[numBigramme])
        temp.translate(table)
        result=temp.strip("#").encode('utf8')
    elif type(chaine)==unicode:
        chaine="#"+chaine+"#"
        for numBigramme,bigramme in enumerate(german2):
            chaine=chaine.replace(bigramme,bdlexique2[numBigramme])
        result=chaine.strip("#").translate(table)
    else:
        result=chaine
    return result

In [16]:
pdLexique["phono"]=pdLexique["phono"].apply(recoder)

In [17]:
with open(nomLexique,'wb') as output:
    pickle.dump(pdLexique, output, pickle.HIGHEST_PROTOCOL)

# TESTS

In [14]:
pdLexique[pdLexique["phono"].str.contains("c")]

,lexeme,phono,case


In [ ]:
mots=pdLexique["phono"].tolist()
letters=set()
for mot in mots:
    letters|=set(mot)
done=set(u"aeäijouöüypbmfvwtdnlrszckqgh")
print ("".join(sorted(letters-done)))

In [ ]:
pdLexique.pivot_table(columns=['case'],index=u"lexeme",values="phono",aggfunc=lambda x:",".join(x)).dropna(thresh=5)

In [ ]:
print recoder(u"cräwem")

In [ ]:
import features
features.add_config('../german.ini')
fs=features.FeatureSystem('phonemes')